# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("../output_data/cities.csv", encoding="utf-8")
cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,sydney mines,46.2367,-60.2177,73.09,51.0,0.0,5.99,CA,1.627586e+09
1,taolanaro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bredasdorp,-34.5322,20.0403,48.27,89.0,8.0,6.76,ZA,1.627586e+09
3,mataura,-46.1927,168.8643,40.48,94.0,87.0,3.89,NZ,1.627586e+09
4,arraial do cabo,-22.9661,-42.0278,64.26,48.0,75.0,23.02,BR,1.627586e+09
...,...,...,...,...,...,...,...,...,...
606,inta,66.0317,60.1659,49.03,80.0,98.0,6.17,RU,1.627586e+09
607,songjianghe,42.1859,127.4790,67.53,97.0,100.0,3.83,CN,1.627586e+09
608,saint-joseph,-21.3667,55.6167,68.38,82.0,43.0,17.31,RE,1.627586e+09
609,cocorit,27.5833,-109.9667,95.34,35.0,21.0,10.42,MX,1.627586e+09


In [3]:
cities = cities.dropna()
cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,sydney mines,46.2367,-60.2177,73.09,51.0,0.0,5.99,CA,1.627586e+09
2,bredasdorp,-34.5322,20.0403,48.27,89.0,8.0,6.76,ZA,1.627586e+09
3,mataura,-46.1927,168.8643,40.48,94.0,87.0,3.89,NZ,1.627586e+09
4,arraial do cabo,-22.9661,-42.0278,64.26,48.0,75.0,23.02,BR,1.627586e+09
5,geraldton,-28.7667,114.6000,65.12,82.0,75.0,20.71,AU,1.627586e+09
...,...,...,...,...,...,...,...,...,...
606,inta,66.0317,60.1659,49.03,80.0,98.0,6.17,RU,1.627586e+09
607,songjianghe,42.1859,127.4790,67.53,97.0,100.0,3.83,CN,1.627586e+09
608,saint-joseph,-21.3667,55.6167,68.38,82.0,43.0,17.31,RE,1.627586e+09
609,cocorit,27.5833,-109.9667,95.34,35.0,21.0,10.42,MX,1.627586e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
humidity = cities["Humidity"]
max_humidity = humidity.max()

In [5]:
locations = cities[["Lat","Lng"]]

###  Please note the Humidity Heat Map screenshot was placed in the Images folder \Python-API-challenge\Images

In [6]:
# Plot Heatmap
fig = gmaps.figure()

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=max_humidity,point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [8]:
ideal_cities = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) 
                          & (cities["Wind Speed"] < 10) & (cities["Cloudiness"] == 0)]
ideal_cities.head()                                                                                                                

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,sydney mines,46.2367,-60.2177,73.09,51.0,0.0,5.99,CA,1.627586e+09
94,shingu,33.7333,135.9833,76.91,84.0,0.0,5.12,JP,1.627586e+09
104,bafra,41.5678,35.9069,73.18,70.0,0.0,4.18,TR,1.627586e+09
168,bay roberts,47.5999,-53.2648,73.65,54.0,0.0,8.99,CA,1.627586e+09
192,jiayuguan,39.8167,98.3000,76.23,19.0,0.0,3.83,CN,1.627586e+09


In [9]:
ideal_cities = ideal_cities.dropna()
ideal_cities.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,sydney mines,46.2367,-60.2177,73.09,51.0,0.0,5.99,CA,1.627586e+09
94,shingu,33.7333,135.9833,76.91,84.0,0.0,5.12,JP,1.627586e+09
104,bafra,41.5678,35.9069,73.18,70.0,0.0,4.18,TR,1.627586e+09
168,bay roberts,47.5999,-53.2648,73.65,54.0,0.0,8.99,CA,1.627586e+09
192,jiayuguan,39.8167,98.3000,76.23,19.0,0.0,3.83,CN,1.627586e+09


In [10]:
ideal_cities.reset_index(inplace=True)
ideal_cities.head()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sydney mines,46.2367,-60.2177,73.09,51.0,0.0,5.99,CA,1.627586e+09
1,94,shingu,33.7333,135.9833,76.91,84.0,0.0,5.12,JP,1.627586e+09
2,104,bafra,41.5678,35.9069,73.18,70.0,0.0,4.18,TR,1.627586e+09
3,168,bay roberts,47.5999,-53.2648,73.65,54.0,0.0,8.99,CA,1.627586e+09
4,192,jiayuguan,39.8167,98.3000,76.23,19.0,0.0,3.83,CN,1.627586e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_list = []

In [12]:
len(ideal_cities)

20

In [13]:
for index, row in ideal_cities.iterrows():
    lat = ideal_cities.loc[index]["Lat"]
    lng = ideal_cities.loc[index]["Lng"]
    
    # Use nearby search with a radius of 5000 for hotels
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
          "location": f'{lat},{lng}',
          "radius": 5000,
          "types": "hotel",
          "key": g_key
         }  
    
    requested = requests.get(base_url, params=params)
    hotel_jsn = requested.json()
#    print(json.dumps(hotel_jsn, indent=4, sort_keys=True))
    try:
          hotel_list.append(hotel_jsn['results'][0]['name'])
    except:
          hotel_list.append("")
                
ideal_cities["Hotel Name"] = hotel_list
ideal_cities = ideal_cities.dropna()


In [14]:
ideal_cities.head()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,sydney mines,46.2367,-60.2177,73.09,51.0,0.0,5.99,CA,1.627586e+09,Sydney Mines
1,94,shingu,33.7333,135.9833,76.91,84.0,0.0,5.12,JP,1.627586e+09,Shingu
2,104,bafra,41.5678,35.9069,73.18,70.0,0.0,4.18,TR,1.627586e+09,Bafra
3,168,bay roberts,47.5999,-53.2648,73.65,54.0,0.0,8.99,CA,1.627586e+09,Bay Roberts
4,192,jiayuguan,39.8167,98.3000,76.23,19.0,0.0,3.83,CN,1.627586e+09,Jiayuguan City


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

In [16]:
hotel_info = [info_box_template.format(**row) for index, row in ideal_cities.iterrows()]


###  Please note the Hotel Map screenshot was placed in the Images folder \Python-API-challenge\Images

In [17]:
# Add marker layer ontop of heat map
# markers = gmaps.marker_layer(locations)
locations = ideal_cities[["Lat", "Lng"]]
fig = gmaps.figure(center=(31.0, 31.0), zoom_level=1.7)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)

#fig = gmaps.figure()
#markers = gmaps.marker_layer(locations)

fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))